仕掛けた時の連鎖数と勝敗の関係を出力

In [1]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import os
import collections

In [2]:
folder = "puyopuyo_saikyo_league/stats/s1_day1_momoken_rein_2"
games = 57

In [4]:
#仕掛けの連鎖数と勝ち負けの出力

#1Pと2Pそれぞれについて勝ち=1, 負け=0のリスト作成(win_nP_list)
win_1P_list = []
win_2P_list = []

for i in range(games):
    #ファイル読み込み
    filepath = folder + "/" + str(i) + "_6.csv"
    df_game = pd.read_csv(filepath)
    
    #勝ちなら1、負けなら0を格納
    if df_game["Win"].iloc[0] == 1:
        win_1P_list.append(1)
        win_2P_list.append(0)
    else:
        win_1P_list.append(0)
        win_2P_list.append(1)      

#各試合について仕掛けのツモ数のリストを作成(shikake_r_list)
#各試合についてどちらが仕掛けたかのリスト(shikake_list)を作成
shikake_r_list = []
shikake_list = []

for i in range(games):
    #ファイル読み込み
    filepath = folder + "/" + str(i) + "_6.csv"
    df_game = pd.read_csv(filepath)
    num = 0
    index = 0
    N = len(df_game["Player"])
    
    #仕掛けとカウントしたらnumを1にして次の試合へ 
    while (num<1):
        #全消し戦はノーカウント(-1)
        if df_game["Zenkeshi"].iloc[0] == 1:
            shikake_r_list.append(-1)
            shikake_list.append(-1)
            num = 1
        #連鎖がない試合の場合ノーカウント(-1)
        elif N == 0:
            shikake_r_list.append(-1)
            shikake_list.append(-1)
            num = 1
         #(★)の後に連鎖が発生せず試合が終わった(index=N)場合、★をカウント
        elif index == N:
            shikake_r_list.append(df_game["Rensa"].iloc[index-1])
            shikake_list.append(df_game["Player"].iloc[index-1])
        else:
            #12手以内の単発でおじゃま発生が4つ未満かつ対応が発生しなかった場合、ノーカウント→次の行を参照→(★)
            if (df_game["Rensa"].iloc[index]==1)&(df_game["Tumo"].iloc[index]<=12)&(df_game["Ojama_detail"].iloc[index]<4)&(df_game["Continue"].iloc[index]==0):
                index += 1
            #ほとんどがこの場合。連鎖数とプレイヤーを格納
            else:
                if df_game["Player"].iloc[index] == 1:
                    shikake_r_list.append(df_game["Rensa"].iloc[index])
                    shikake_list.append(df_game["Player"].iloc[index])
                else:
                    shikake_r_list.append(df_game["Rensa"].iloc[index])
                    shikake_list.append(df_game["Player"].iloc[index])
                num = 1


#(1P/2P)/(勝/負)の4パターンで連鎖数を分類
rensa_win_1P = []
rensa_win_2P = []
rensa_lose_1P = []
rensa_lose_2P = []

for i in range(len(win_1P_list)):
    if shikake_list[i] == 1:
        if win_1P_list[i] == 1:
            rensa_win_1P.append(shikake_r_list[i])
        else:
            rensa_lose_1P.append(shikake_r_list[i])
    elif shikake_list[i] == 2:
        if win_2P_list[i] == 1:
            rensa_win_2P.append(shikake_r_list[i])
        else:
            rensa_lose_2P.append(shikake_r_list[i])
    else:
        pass
    
#連鎖数ごとの合計を出力
w_1P = []
l_1P = []
w_2P = []
l_2P = []
index_list = []

for i in range(1,20):
    w_1P.append(sum(x==i for x in rensa_win_1P))
    l_1P.append(sum(x==i for x in rensa_lose_1P))
    w_2P.append(sum(x==i for x in rensa_win_2P))
    l_2P.append(sum(x==i for x in rensa_lose_2P))
    index_list.append(str(i) + "連鎖")
    

df_shikake_rensa = pd.DataFrame(list(zip(w_1P, l_1P, w_2P, l_2P)), columns=["1P勝利", "1P敗北", "2P勝利", "2P敗北"],
                               index=index_list)   
df_shikake_rensa

,1P勝利,1P敗北,2P勝利,2P敗北
1連鎖,5,10,6,9
2連鎖,3,1,3,5
3連鎖,1,0,1,0
4連鎖,0,0,0,0
5連鎖,1,0,0,1
6連鎖,0,0,0,0
7連鎖,0,0,0,0
8連鎖,0,0,0,0
9連鎖,0,0,0,0
10連鎖,0,0,0,0
